<a href="https://colab.research.google.com/github/samyff12/LATAM/blob/main/IMF_BM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
! pip install wbgapi
import wbgapi as wb  # api importe de datos del WB
import pandas as pd
import numpy as np

In [30]:
wb.source.info(27)   # global economic prospects 27 INFO

id,name,code,concepts,lastupdated
27,Global Economic Prospects,GEP,3,2022-01-11
,1 elements,,,


In [83]:
# lista de paises de LATAM
latam = list(dict(wb.economy.coder(['Latin America and the Caribbean','Argentina', 'Peru', 'Chile', 'Dominican Republic',
                  'Panama','Colombia','El Salvador','Guatemala','Mexico','Nicaragua','Bolivia',
                  'Costa Rica','Brazil','Honduras','Paraguay','Ecuador','Uruguay']))
.values())

latam.append('LCN') # añadiendo AL y el Caribe

data = wb.data.DataFrame('NYGDPMKTPKDZ', latam , db=27).sort_values('2021',ascending=False)
data

,2019,2020,2021,2022,2023
economy,,,,,
PER,2.2,-11.1,13.2,3.2,3.0
CHL,0.9,-5.8,11.8,2.2,1.8
DOM,5.1,-6.8,10.8,5.0,5.0
ARG,-2.0,-9.9,10.0,2.6,2.1
COL,3.3,-6.8,9.9,4.1,3.5
PAN,3.0,-17.9,9.9,7.8,5.0
SLV,2.6,-7.9,8.0,4.0,2.5
GTM,3.9,-1.5,7.6,3.9,3.5
LCN,0.8,-6.4,6.7,2.6,2.7


In [89]:
data['21/19'] = ((100*(1+data['2020']/100))*(1+data['2021']/100) - 100).round(1)
data.sort_values('21/19',ascending=False)

,2019,2020,2021,2022,2023,21/19
economy,,,,,,
GTM,3.9,-1.5,7.6,3.9,3.5,6.0
CHL,0.9,-5.8,11.8,2.2,1.8,5.3
PRY,-0.4,-0.8,4.3,4.0,3.9,3.5
NIC,-3.7,-2.0,5.5,3.0,2.5,3.4
DOM,5.1,-6.8,10.8,5.0,5.0,3.3
COL,3.3,-6.8,9.9,4.1,3.5,2.4
BRA,1.2,-3.9,4.9,1.4,2.7,0.8
CRI,2.3,-4.1,5.0,3.5,3.2,0.7
PER,2.2,-11.1,13.2,3.2,3.0,0.6


In [ ]:
import requests # Python 3.6

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/IFS/M.GB.PMP_IX' # adjust codes here

# Navigate to series in API-returned JSON data
data = (requests.get(f'{url}{key}').json()
        ['CompactData']['DataSet']['Series'])

print(data['Obs'][-1]) # Print latest observation

In [ ]:
import requests # Python 3.6

url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/'
key = 'CompactData/WEO/NGDP_RPCH' # adjust codes here

# Navigate to series in API-returned JSON data
data = (requests.get(f'{url}{key}').json()
        ['CompactData']['DataSet']['Series'])

print(data['Obs'][-1]) # Print latest observation

In [ ]:
growth = 'NGDP_RPCH'
gdp_const = 'NGDP'

In [ ]:
import weo
from weo import WEO


weo.all_releases()[-1][0]            # ultima publicacion

weo.download(year = weo.all_releases()[-1][0],
             release = weo.all_releases()[-1][1],
             filename="weo.csv")

w = WEO("weo.csv")


In [107]:
weo.download(year=2021, release=2, filename="weo1.csv")

weo1.csv 0.1Mb
Downloaded 2021-Oct WEO dataset


('weo1.csv',
 'https://www.imf.org/-/media/Files/Publications/WEO/WEO-Database/2021/02/WEOOct2021all.xls')

In [ ]:
import requests 

# URL for the IMF JSON Restful Web Service, 
# IFS database, and Australian export prices series
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/Q.AU.PXP_IX.?startPeriod=1957&endPeriod=2016'

# Get data from the above URL using the requests package
data = requests.get(url).json()

# Load data into a pandas dataframe
auxp = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs'])

# Show the last five observiations
auxp.tail()